In [12]:

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MultiLabelBinarizer

df = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")

'''
Teste 1:
print(df.head())
#Verificando se existem valores nulos
print(df.isnull().sum())
#Verificando se existem valores duplicados
print("Duplicados:", df.duplicated().sum())
'''


#Se a nota ≥ 4 (curtiu).
#Caso contrário (não curtiu).
#Converter notas para escala binária: 

ratings['binary_rating'] = (ratings['rating'] >= 4).astype(int)

# Juntar avaliações com os metadados dos filmes
df = pd.merge(ratings, movies, on="movieId", how="left")

# Mapear IDs para índices 0..N
user_ids = sorted(ratings['userId'].unique())
item_ids = sorted(ratings['movieId'].unique())
u2idx = {u:i for i,u in enumerate(user_ids)}
i2idx = {i:j for j,i in enumerate(item_ids)}
idx2i = {v:k for k,v in i2idx.items()}

n_users = len(user_ids)
n_items = len(item_ids)

# Criar matriz binária usuário-item
rows = ratings['userId'].map(u2idx).to_numpy()
cols = ratings['movieId'].map(i2idx).to_numpy()
data = ratings['binary_rating'].to_numpy()
M = csr_matrix((data, (rows, cols)), shape=(n_users, n_items))

# Calcular similaridade entre itens
item_matrix = M.T  # itens x usuários
item_matrix_norm = normalize(item_matrix, axis=1)


# Função para gerar recomendações Item-based

def recommend_item_based_ondemand(user_index, topk=10):
    seen = set(M[user_index].nonzero()[1])  # itens já vistos
    scores = np.zeros(M.shape[1])

    # Para cada item que o usuário curtiu
    for i in seen:
        scores += item_matrix_norm[i].dot(item_matrix_norm.T).toarray().ravel()

    # Não recomendar itens já vistos
    scores[list(seen)] = -1
    recs = np.argsort(-scores)[:topk]
    return recs
    
# Transformar gêneros em lista
    movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
    
# One-hot encoding dos gêneros
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(movies['genres'])
genres_df = pd.DataFrame(genres_encoded, index=movies['movieId'], columns=mlb.classes_)

# Função de recomendação baseado em conteúdo
def recommend_content_based(user_id, topk=10):
    # Filmes que o usuário curtiu
    liked_movies = ratings[(ratings['userId'] == user_id) & (ratings['binary_rating'] == 1)]['movieId'].tolist()
    if not liked_movies:
        return []  # usuário sem histórico
    
    # Similaridade cosseno com todos os filmes
    liked_vectors = genres_df.loc[liked_movies].values
    all_vectors = genres_df.values
    sim = cosine_similarity(liked_vectors, all_vectors).sum(axis=0)
    
    # Remover filmes já curtidos
    liked_idx = [genres_df.index.get_loc(m) for m in liked_movies]
    sim[liked_idx] = -1
    
    # Top K filmes
    top_indices = np.argsort(-sim)[:topk]
    recommended_movieIds = genres_df.index[top_indices].tolist()
    
    # Retornar títulos
    return movies[movies['movieId'].isin(recommended_movieIds)]['title'].tolist()

    # Gerar recomendações para 2 usuários aleatórios
sample_users = np.random.choice(range(n_users), size=5, replace=False)

for u in sample_users:
    rec_indices = recommend_item_based_ondemand(u, topk=5)
    recommended_movieIds = [idx2i[i] for i in rec_indices]
    recommended_titles = movies[movies['movieId'].isin(recommended_movieIds)]['title'].tolist()
    print(f"Usuário {user_ids[u]} - Recomendações Item-based:")
    for t in recommended_titles:
        print("  ", t)
    print("\n")

for u in sample_users:
    recs = recommend_content_based(u, topk=5)
    print(f"Usuário {user_ids[u]} - Recomendações Baseado em Conteúdo:")
    for t in recs:
        print("  ", t)
    print("\n")



Usuário 147690 - Recomendações Item-based:
   Usual Suspects, The (1995)
   Reservoir Dogs (1992)
   Monty Python and the Holy Grail (1975)
   Big Lebowski, The (1998)
   Lord of the Rings: The Fellowship of the Ring, The (2001)


Usuário 167195 - Recomendações Item-based:
   Godfather, The (1972)
   Star Wars: Episode V - The Empire Strikes Back (1980)
   Matrix, The (1999)
   Memento (2000)
   Lord of the Rings: The Fellowship of the Ring, The (2001)


Usuário 168966 - Recomendações Item-based:
   Crimson Tide (1995)
   Forrest Gump (1994)
   Speed (1994)
   Fugitive, The (1993)
   Terminator 2: Judgment Day (1991)


Usuário 92517 - Recomendações Item-based:
   Bourne Identity, The (2002)
   Batman Begins (2005)
   300 (2007)
   Dark Knight, The (2008)
   Iron Man (2008)


Usuário 143331 - Recomendações Item-based:
   Toy Story (1995)
   Star Wars: Episode IV - A New Hope (1977)
   Star Wars: Episode V - The Empire Strikes Back (1980)
   Back to the Future (1985)
   Ghostbusters (a.k